In [7]:
# import library
import re
import sys
import os
import shutil
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time
import pandas as pd

In [2]:
# Ambil file asli sebanyak 200
directory_path_real = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_asli'

files_real = os.listdir(directory_path_real)
list_files_real = []
counter = 0

# Ambil daftar nama file
for file in files_real:
    if '.DS_Store' not in file:
        list_files_real.append(file)
        
all_text_real = ""
for file in list_files_real:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_asli/' + file, 'r') as fileNow:
        content = fileNow.read()
        lines = content.splitlines()
        if counter < 200:
            counter += 1
            # sumber file
            source_file = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_asli/' + file
            
            # file tujuan
            destination_folder = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/ToolsExploration/Data/Data_asli'
            
            # menyalin file ke folder baru
            shutil.copy(source_file, destination_folder)


In [3]:
# Ambil file parafrasa sebanyak 200
directory_path_paraphrased = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_parafrasa'

files_paraphrased = os.listdir(directory_path_paraphrased)
list_files_paraphrased = []
counter = 0

# Ambil daftar nama file
for file in files_paraphrased:
    if '.DS_Store' not in file:
        list_files_paraphrased.append(file)
        
all_text_paraphrased = ""
for file in list_files_paraphrased:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_parafrasa/' + file, 'r') as fileNow:
        content = fileNow.read()
        lines = content.splitlines()
        if counter < 200:
            counter += 1
            # sumber file
            source_file = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_parafrasa/' + file
            
            # file tujuan
            destination_folder = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/ToolsExploration/Data/Data_parafrasa'
            
            # menyalin file ke folder baru
            shutil.copy(source_file, destination_folder)
        
        


In [9]:
# method untuk menghitung nilai relatedness
def relatedness(model):
    # Baca file rw untuk evaluasi word embedding dengan teknik relatedness cbow
    dir_file = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataEvaluasiGitHub/word-similarity/monolingual/en/rw.csv'
    list_model_similarity = []
    words_exist = []
    words_not_exist = []
    list_human_score = []
    
    df = pd.read_csv(dir_file)
    list_human_score = []
    
    for index, row in df.iterrows():
        word_1 = row["word1"].lower()
        word_2 = row["word2"].lower()
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model.wv[word_1])
            
            if word_1 not in words_exist:
                words_exist.append(word_1)
            
            wv_2.append(model.wv[word_2])
            
            if word_2 not in words_exist:
                words_exist.append(word_2)
                
            similarity = model.wv.similarity(word_1, word_2)
            list_model_similarity.append(similarity)
            
            human_score = float(row["similarity"])
            list_human_score.append(human_score)
            
        except:
            if word_1 in words_exist: # kata 2 yang tidak ada di model
                if word_2 not in words_not_exist:
                    words_not_exist.append(word_2)
            
            if word_2 in words_exist: # kata 1 yang tidak ada di model
                if word_1 not in words_not_exist:
                    words_not_exist.append(word_1)
        
    
   
        

    rho_model, p_model = spearmanr(list_model_similarity, list_human_score)
    print("banyak kata ada di model", len(words_exist))
    print("banyak kata tidak ada di model", len(words_not_exist))
    return rho_model


In [10]:
# method untuk menilai model dengan teknik analogi kata
def word_analogy(model):
    # Baca file word_analogy untuk evaluasi word embedding dengan teknik word analogy (menggunakan library)

    dir_file = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataEvaluasiGitHub/word-analogy/monolingual/en/semeval.csv'
    df = pd.read_csv(dir_file)
    true_predicted = 0
    false_predicted = 0
    words_exist = []
    words_not_exist = []
    
    for index, row in df.iterrows():
        word_1 = row["word1"].lower()
        word_2 = row["word2"].lower()
        word_3 = row["word3"].lower()
        word_4 = row["target"].lower()
        word_predicted_curr = ""
        try:
            word_predicted_curr = model.wv.most_similar(positive=[word_2, word_3], negative=[word_1], topn=1)[0][0]
         
            if word_predicted_curr == word_4:
                true_predicted += 1
            else:
                false_predicted += 1
            
            if word_1 not in words_exist:
                words_exist.append(word_1)
            
            if word_2 not in words_exist:
                words_exist.append(word_2)
            
            if word_3 not in words_exist:
                words_exist.append(word_3)
            
            if word_4 not in words_exist:
                words_exist.append(word_4)

        except:
            continue
            
        # untuk menambahkan word_1 jika tidak ada di model    
        try:
            word_vector = model.wv[word_1]
        except:
            if word_1 not in words_not_exist:
                    words_not_exist.append(word_1)
        
        # untuk menambahkan word_2 jika tidak ada di model    
        try:
            word_vector = model.wv[word_2]
        except:
            if word_2 not in words_not_exist:
                    words_not_exist.append(word_2)

        # untuk menambahkan word_3 jika tidak ada di model    
        try:
            word_vector = model.wv[word_3]
        except:
            if word_3 not in words_not_exist:
                    words_not_exist.append(word_3)
        
        # untuk menambahkan word_4 jika tidak ada di model    
        try:
            word_vector = model.wv[word_4]
        except:
            if word_4 not in words_not_exist:
                    words_not_exist.append(word_4)

    print("true_predicted", true_predicted, true_predicted / (true_predicted+false_predicted))
    print("false_predicted", false_predicted, false_predicted / (true_predicted+false_predicted))
    print("banyak kata ada di model", len(words_exist))
    print("banyak kata tidak ada di model", len(words_not_exist))

In [2]:
# Ambil file asli dari folder
directory_path_real = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/ToolsExploration/Data/Data_asli'

files_real = os.listdir(directory_path_real)
list_files_real = []

# Ambil daftar nama file
for file in files_real:
    if '.DS_Store' not in file:
        list_files_real.append(file)
        
all_text_real = ""
for file in list_files_real:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/ToolsExploration/Data/Data_asli/' + file, 'r') as fileNow:
        content = fileNow.read()
        all_text_real += content


In [3]:
# Ambil file parafrasa dari folder
directory_path_paraphrased = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/ToolsExploration/Data/Data_parafrasa'

files_paraphrased = os.listdir(directory_path_paraphrased)
list_files_paraphrased = []

# Ambil daftar nama file
for file in files_paraphrased:
    if '.DS_Store' not in file:
        list_files_paraphrased.append(file)
        
all_text_paraphrased = ""
for file in list_files_paraphrased:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/ToolsExploration/Data/Data_parafrasa/' + file, 'r') as fileNow:
        content = fileNow.read()
        all_text_paraphrased += content
        


In [4]:
# Melakukan lowercase lagi
all_text_real = all_text_real.lower()
all_text_paraphrased = all_text_paraphrased.lower()

In [5]:
# Memisahkan text per kalimat (menggunakan delimiter titik)
list_of_sentences_real = []
list_of_sentences_paraphrased = []

list_of_sentences_real = all_text_real.split('.')
list_of_sentences_paraphrased = all_text_paraphrased.split('.')

print(list_of_sentences_real[0:10])
print(list_of_sentences_paraphrased[0:10])

['computer-aided structure-activity based prediction studies in drug design helps to treat diseases with novel biomarkers', ' prediction of activity spectra for substances(pass) database comprised 46,000 biologically well-known active drugs and screening are performed before the establishment of an in vitro experiment', ' pass gives the significant bioactivities of chemical compounds as pa(probable activity) and pi(probable inactivity) values to mention the compounds, whether they are active are inactive', ' the pa values higher than 0', '7 indicated this compound would be active in experiment and pi values indicate theirs inactivate possibilities', ' the admetsar chemoinformatics based tool used to predict absorption, metabolism, excretion, and toxicity of the particular compound', ' based on these criteria, the outcomes of an in vitro experiment will lower the risk of negative results[13,14]', '\n\n\n\nclc-pred tools performed to predict cytotoxicity of tumor cell lines, and it is ba

In [8]:
# melakukan tokenisasi dan menggabungkan teks asli dan parafrasa


list_of_sentences_for_model = []

tokenizer = RegexpTokenizer(r'[a-zA-Z]+')

for sentence in list_of_sentences_real:
    sentence_tokenized = tokenizer.tokenize(sentence)
    list_of_sentences_for_model.append(sentence_tokenized)

for sentence in list_of_sentences_paraphrased:
    sentence_tokenized = tokenizer.tokenize(sentence)
    list_of_sentences_for_model.append(sentence_tokenized)

print(list_of_sentences_for_model[0:10])


[['computer', 'aided', 'structure', 'activity', 'based', 'prediction', 'studies', 'in', 'drug', 'design', 'helps', 'to', 'treat', 'diseases', 'with', 'novel', 'biomarkers'], ['prediction', 'of', 'activity', 'spectra', 'for', 'substances', 'pass', 'database', 'comprised', 'biologically', 'well', 'known', 'active', 'drugs', 'and', 'screening', 'are', 'performed', 'before', 'the', 'establishment', 'of', 'an', 'in', 'vitro', 'experiment'], ['pass', 'gives', 'the', 'significant', 'bioactivities', 'of', 'chemical', 'compounds', 'as', 'pa', 'probable', 'activity', 'and', 'pi', 'probable', 'inactivity', 'values', 'to', 'mention', 'the', 'compounds', 'whether', 'they', 'are', 'active', 'are', 'inactive'], ['the', 'pa', 'values', 'higher', 'than'], ['indicated', 'this', 'compound', 'would', 'be', 'active', 'in', 'experiment', 'and', 'pi', 'values', 'indicate', 'theirs', 'inactivate', 'possibilities'], ['the', 'admetsar', 'chemoinformatics', 'based', 'tool', 'used', 'to', 'predict', 'absorption',

In [11]:
# Membuat model word embedding dengan teknik word2vec dengan cbow 1

# waktu mulai
start_time = time.time()

model = Word2Vec(sentences=list_of_sentences_for_model, vector_size=10, min_count=1, workers = 10, compute_loss=True,  window=5, sg = 0, cbow_mean = 0, alpha = 0.01, seed = 10, epochs = 100)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")



Total training time: 0 hours 0 minutes 14 seconds


In [108]:
# save model word embedding dengan teknik word2vec dengan cbow 1
model.save("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_1.model")


In [9]:
# load model word embedding dengan teknik word2vec dengan cbow 1
model = Word2Vec.load("/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/ModelWord2vecSetelahPembagianData/word2vec_cbow_1.model")